### Project Explanation

In this project I have 2 CSVs
* routes.csv
    * Contains all airplane routes from a single year
* airports.csv
    * Contains all worldwide airports

The goal of this project is to mimic a route optimization model where a user can specify: 
* Starting point
* Ending point
* Time constraint

### Notebook Setup

In [15]:
# pip install graphdatascience
import numpy as np 
import pandas as pd
import collections
import os
import math
from yfiles_jupyter_graphs import GraphWidget
from itertools import permutations
import itertools
from neo4j import GraphDatabase

pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)

### Neo4j Settings

In [3]:
from graphdatascience import GraphDataScience

HOST = 'neo4j://35.237.xxx.xxx'
USERNAME = 'neo4j'
PASSWORD = 'password'
DATABASE = 'routes'

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=False)
gds.set_database(DATABASE)

# Getting driver set up for yFiles
driver = GraphDatabase.driver(uri=HOST,auth=(USERNAME, PASSWORD),database='routes')
session = driver.session(database = 'routes')

### Part 1: Explore Data & Data Cleaning

In [4]:
# Import route data
route_cols = ['airline', 'airlineID', 'source', 'sourceAirportID',
              'dest', 'destAirportID', 'codeshare', 'stops', 'equipment']
routes_df = pd.read_csv("routes/routes.csv", skiprows=1, names = route_cols)
routes_df['sourceAirportID'] = pd.to_numeric(routes_df['sourceAirportID'].astype(str), 'coerce')
routes_df['destAirportID'] = pd.to_numeric(routes_df['destAirportID'].astype(str), 'coerce')
    
print(routes_df.shape)
routes_df.head()

(67663, 9)


,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [5]:
# Data stats
print(routes_df.shape)
print(routes_df.head())
print(routes_df.isna().sum())
print(routes_df['stops'].value_counts())
routes_df.describe(include='all')


(67663, 9)
  airline airlineID source  sourceAirportID dest  destAirportID codeshare  \
0      2B       410    AER           2965.0  KZN         2990.0       NaN   
1      2B       410    ASF           2966.0  KZN         2990.0       NaN   
2      2B       410    ASF           2966.0  MRV         2962.0       NaN   
3      2B       410    CEK           2968.0  KZN         2990.0       NaN   
4      2B       410    CEK           2968.0  OVB         4078.0       NaN   

   stops equipment  
0      0       CR2  
1      0       CR2  
2      0       CR2  
3      0       CR2  
4      0       CR2  
airline                0
airlineID              0
source                 0
sourceAirportID      220
dest                   0
destAirportID        221
codeshare          53066
stops                  0
equipment             18
dtype: int64
0    67652
1       11
Name: stops, dtype: int64


,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment
count,67663,67663,67663,67443.000000,67663,67442.000000,14597,67663.000000,67645
unique,568,548,3409,NaN,3418,NaN,1,NaN,3945
top,FR,4296,ATL,NaN,ATL,NaN,Y,NaN,320
freq,2484,2484,915,NaN,911,NaN,14597,NaN,9180
mean,NaN,NaN,NaN,2693.690079,NaN,2695.737760,NaN,0.000163,NaN
std,NaN,NaN,NaN,1672.391798,NaN,1673.306719,NaN,0.012749,NaN
min,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,0.000000,NaN
25%,NaN,NaN,NaN,1382.000000,NaN,1382.000000,NaN,0.000000,NaN
50%,NaN,NaN,NaN,2964.000000,NaN,2964.500000,NaN,0.000000,NaN
75%,NaN,NaN,NaN,3670.000000,NaN,3670.000000,NaN,0.000000,NaN


In [6]:
routes_df.head()

,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [7]:
routes_df_explore = routes_df
routes_df_explore['source_dest'] = routes_df_explore['source'] + routes_df_explore['dest']
routes_df_explore['source_dest'].value_counts().head()

ORDATL    20
ATLORD    19
ORDMSY    13
HKTBKK    13
HKGBKK    12
Name: source_dest, dtype: int64

As you can see above that we do in fact have repeat trips in this dataset. This means we will need to give each flight an ID to create new trip relationships.

In [8]:
# Create ID column
routes_df['flightID'] = routes_df.index +1
routes_df.head()

,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment,source_dest,flightID
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,AERKZN,1
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,ASFKZN,2
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2,ASFMRV,3
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2,CEKKZN,4
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2,CEKOVB,5


In [9]:
# Import airport data
airport_df = pd.read_csv("routes/airports.csv")

In [10]:
# Data stats
print(airport_df.shape)
print(airport_df.head())
print(airport_df.isna().sum())
airport_df.describe(include='all')

(7698, 14)
   Airport ID                                         Name          City  \
0           1                               Goroka Airport        Goroka   
1           2                               Madang Airport        Madang   
2           3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3           4                               Nadzab Airport        Nadzab   
4           5  Port Moresby Jacksons International Airport  Port Moresby   

            Country IATA  ICAO  Latitude   Longitude  Altitude Timezone DST  \
0  Papua New Guinea  GKA  AYGA -6.081690  145.391998      5282       10   U   
1  Papua New Guinea  MAG  AYMD -5.207080  145.789001        20       10   U   
2  Papua New Guinea  HGU  AYMH -5.826790  144.296005      5388       10   U   
3  Papua New Guinea  LAE  AYNZ -6.569803  146.725977       239       10   U   
4  Papua New Guinea  POM  AYPY -9.443380  147.220001       146       10   U   

      Database Timezone     Type       Source  
0  Pacifi

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Database Timezone,Type,Source
count,7698.000000,7698,7649,7698,7698,7698,7698.000000,7698.000000,7698.000000,7698,7698,7698,7698,7698
unique,NaN,7658,6955,237,6073,7698,NaN,NaN,NaN,41,8,308,1,1
top,NaN,San Pedro Airport,London,United States,\N,AYGA,NaN,NaN,NaN,1,U,\N,airport,OurAirports
freq,NaN,3,9,1512,1626,1,NaN,NaN,NaN,1184,1862,1021,7698,7698
mean,5170.950117,NaN,NaN,NaN,NaN,NaN,25.808442,-1.390546,1015.873344,NaN,NaN,NaN,NaN,NaN
std,3777.259930,NaN,NaN,NaN,NaN,NaN,28.404946,86.519162,1628.775132,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,-90.000000,-179.876999,-1266.000000,NaN,NaN,NaN,NaN,NaN
25%,1993.250000,NaN,NaN,NaN,NaN,NaN,6.907983,-78.974699,63.000000,NaN,NaN,NaN,NaN,NaN
50%,4068.000000,NaN,NaN,NaN,NaN,NaN,34.085701,6.375653,352.000000,NaN,NaN,NaN,NaN,NaN
75%,7728.750000,NaN,NaN,NaN,NaN,NaN,47.239551,56.000599,1203.000000,NaN,NaN,NaN,NaN,NaN


In [11]:
# Drop airport that don't have IATA data
airport_df = airport_df[airport_df.IATA != '\\N']
print(airport_df.shape)
airport_df.tail()

(6072, 14)


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Database Timezone,Type,Source
7684,14097,Bilogai-Sugapa Airport,Sugapa-Papua Island,Indonesia,UGU,WABV,-3.739560,137.031998,7348,\N,\N,\N,airport,OurAirports
7687,14100,Ramon Airport,Eilat,Israel,ETM,LLER,29.723694,35.011416,288,\N,\N,\N,airport,OurAirports
7688,14101,Rustaq Airport,Al Masna'ah,Oman,MNH,OORQ,23.640556,57.487500,349,\N,\N,\N,airport,OurAirports
7689,14102,Laguindingan Airport,Cagayan de Oro City,Philippines,CGY,RPMY,8.612203,124.456496,190,\N,\N,\N,airport,OurAirports
7696,14109,Desierto de Atacama Airport,Copiapo,Chile,CPO,SCAT,-27.261200,-70.779198,670,\N,\N,\N,airport,OurAirports


In [12]:
# make new route df with route count info
routes_all = pd.DataFrame(routes_df.groupby(['source', 'dest']).size().reset_index(name='counts'))

airport_all = airport_df[['Name','City','Country','Latitude', 'Longitude', 'IATA']]
IATA_array = airport_all["IATA"].tolist()

In [13]:
airport_all.head()

,Name,City,Country,Latitude,Longitude,IATA
0,Goroka Airport,Goroka,Papua New Guinea,-6.081690,145.391998,GKA
1,Madang Airport,Madang,Papua New Guinea,-5.207080,145.789001,MAG
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,-5.826790,144.296005,HGU
3,Nadzab Airport,Nadzab,Papua New Guinea,-6.569803,146.725977,LAE
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,-9.443380,147.220001,POM


In [14]:
# only keep route with airports that have IATA code
routes_all = routes_all[routes_all['source'].isin(IATA_array)]
routes_all = routes_all[routes_all['dest'].isin(IATA_array)]

In [15]:
# only keep route with airports that have IATA code
routes_df = routes_df[routes_df['source'].isin(IATA_array)]
routes_df = routes_df[routes_df['dest'].isin(IATA_array)]
print(routes_df.shape)
routes_df.head()

(66934, 11)


,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment,source_dest,flightID
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,AERKZN,1
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,ASFKZN,2
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2,ASFMRV,3
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2,CEKKZN,4
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2,CEKOVB,5


In [16]:
airport_df.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Database Timezone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [17]:
# Get the dataframe to calculate distance for all routes
lat_long_df = routes_df[['source', 'dest']]
merge_df = pd.merge(lat_long_df, airport_df[['IATA', 'Latitude', 'Longitude']], left_on = 'source', right_on = 'IATA')
merge_df = merge_df.rename(columns = {'Latitude': 'source_lat', 'Longitude': 'source_long'})
merge_df_2 = pd.merge(merge_df, airport_df[['IATA', 'Latitude', 'Longitude']], left_on = 'dest', right_on = 'IATA')
merge_df_2 = merge_df_2.rename(columns = {'Latitude': 'dest_lat', 'Longitude': 'dest_long'})
lat_long_df = merge_df_2.drop(['IATA_x', 'IATA_y'], axis = 1)
lat_long_df.head()


,source,dest,source_lat,source_long,dest_lat,dest_long
0,AER,KZN,43.449902,39.956600,55.606201,49.278702
1,ASF,KZN,46.283298,48.006302,55.606201,49.278702
2,CEK,KZN,55.305801,61.503300,55.606201,49.278702
3,DME,KZN,55.408798,37.906300,55.606201,49.278702
4,DME,KZN,55.408798,37.906300,55.606201,49.278702


In [18]:
# add distance column
distances = []

for index, row in lat_long_df.iterrows():
    R = 6371
    phi1 = math.radians(row['source_lat'])
    phi2 = math.radians(row['dest_lat'])
    delta_phi = math.radians(row['dest_lat'] - row['source_lat'])
    delta_lambda = math.radians(row['dest_long'] - row['source_long'])
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    distance = round(distance * 0.621371,1)
    distances.append(distance)

lat_long_df['distance'] = distances
lat_long_df['source_dest'] = lat_long_df['source'] + lat_long_df['dest']

lat_long_df.head()
lat_long_df_dup = lat_long_df.drop_duplicates()


In [19]:
# Add distance column to routes_df
routes_df = pd.merge(routes_df, lat_long_df_dup[['source_dest', 'distance']], left_on = 'source_dest', right_on = 'source_dest')
print(routes_df.shape)
routes_df.head()

(66934, 12)


,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment,source_dest,flightID,distance
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,AERKZN,1,936.3
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,ASFKZN,2,646.5
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2,ASFMRV,3,278.5
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2,CEKKZN,4,478.8
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2,CEKOVB,5,831.8


In [20]:
# Add a revenue column (value) - assuming $0.70/mile
routes_df['value'] = routes_df['distance'] * 0.70
routes_df.head()

,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment,source_dest,flightID,distance,value
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,AERKZN,1,936.3,655.41
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,ASFKZN,2,646.5,452.55
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2,ASFMRV,3,278.5,194.95
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2,CEKKZN,4,478.8,335.16
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2,CEKOVB,5,831.8,582.26


In [21]:
# Add a time cost for route value - assuming average speed of 500 MPH
routes_df['duration'] = routes_df['distance'] / 500
routes_df.head()

,airline,airlineID,source,sourceAirportID,dest,destAirportID,codeshare,stops,equipment,source_dest,flightID,distance,value,duration
0,2B,410,AER,2965.0,KZN,2990.0,NaN,0,CR2,AERKZN,1,936.3,655.41,1.8726
1,2B,410,ASF,2966.0,KZN,2990.0,NaN,0,CR2,ASFKZN,2,646.5,452.55,1.2930
2,2B,410,ASF,2966.0,MRV,2962.0,NaN,0,CR2,ASFMRV,3,278.5,194.95,0.5570
3,2B,410,CEK,2968.0,KZN,2990.0,NaN,0,CR2,CEKKZN,4,478.8,335.16,0.9576
4,2B,410,CEK,2968.0,OVB,4078.0,NaN,0,CR2,CEKOVB,5,831.8,582.26,1.6636


In [22]:
routes_df.shape

(66934, 14)

### Part 2: Load Data into Graph

In [23]:
# Loading airports into graph
gds.run_cypher('''
    UNWIND $airports as line
    MERGE (a:Airport {code: line.IATA})
    ON CREATE SET 
        a.name = line.Name, 
        a.city = line.City, 
        a.country = line.Country, 
        a.latitude = line.Latitude, 
        a.longitude = line.Longitude
''',
params = {'airports': airport_df.to_dict(orient='records')}
)

(No data)

In [24]:
# Load flights into graph
gds.run_cypher('''
    UNWIND $routes as line
    MATCH (source:Airport {code: line.source})
    MATCH (destination:Airport {code: line.dest})
    WHERE source.code = line.source
    AND destination.code = line.dest
    MERGE (source)-[f:FLEW_TO {id: line.flightID}]->(destination)
    ON CREATE SET
        f.airline = line.airline,
        f.airlineID = line.airlineID,
        f.distance = line.distance,
        f.value = line.value,
        f.duration = line.duration
''',
params = {'routes': routes_df.to_dict(orient='records')}
)

(No data)

### Part 3: Graph Data Science

#### Shortest Path with Distance

In [25]:
# Create projection to find shortest path with distance
# Used the aggregation of 'MIN' to get the paths into a single

gds.run_cypher('''
    CALL gds.graph.project(
        'shortestPathGraph',
        'Airport',
        {
            FLEW_TO: {
                properties:{
                    distance: {
                        aggregation: 'MIN'
                    }
                }
            }
        }
    )
    YIELD
        graphName AS graph,
        relationshipProjection AS readProjection,
        nodeCount AS nodes,
        relationshipCount AS rels
''')

,graph,readProjection,nodes,rels
0,shortestPathGraph,"{'FLEW_TO': {'orientation': 'NATURAL', 'indexInverse': False, 'aggregation': 'DEFAULT', 'type': 'FLEW_TO', 'properties': {'distance': {'defaultValue': None, 'property': 'distance', 'aggregation': 'MIN'}}}}",6072,37042


In [4]:
# Looking at the shortest path between DFW and YYZ
# Using yens to get the top 5 shortest paths between the two
gds.run_cypher('''
    MATCH (source:Airport {code: 'DFW'}), (dest:Airport {code: 'YYZ'})
    CALL gds.shortestPath.yens.stream('shortestPathGraph',{
        sourceNode: source,
        targetNode: dest,
        k: 5,
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        index,
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost AS totalDistance,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS nodeNames,
        costs as distances
    ORDER BY index 
''')

,index,sourceNodeName,targetNodeName,totalDistance,nodeNames,distances
0,0,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1198.3,"[DFW, YYZ]","[0.0, 1198.3]"
1,1,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.5,"[DFW, DTW, YYZ]","[0.0, 985.8, 1199.5]"
2,2,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.7,"[DFW, IND, YYZ]","[0.0, 760.6, 1199.7]"
3,3,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1200.0,"[DFW, FWA, DTW, YYZ]","[0.0, 858.5, 986.3, 1200.0]"
4,4,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1203.2,"[DFW, STL, YYZ]","[0.0, 550.1, 1203.2]"


In [26]:
# Plotting graph

spGraph = session.run('''
        MATCH (source:Airport {code: 'DFW'}), (dest:Airport {code: 'YYZ'})
        CALL gds.shortestPath.yens.stream('shortestPathGraph',{
        sourceNode: source,
        targetNode: dest,
        k: 5,
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        path
''')


GraphWidget(graph = result.graph())


GraphWidget(layout=Layout(height='500px', width='100%'))

This is interesting to see the shortest path in terms of distance but for this use case we are trying to maximize revenue subject to a time constraint. 

We need to add the following to the model:
* A revenue value for each route
* A time cost for each route
* The ability to string together routes since multiple legs will be desired

In [99]:
# Adding in value and duration as items in the results
gds.run_cypher('''
    MATCH (source:Airport {code: 'DFW'}), (dest:Airport {code: 'YYZ'})
    CALL gds.shortestPath.yens.stream('shortestPathGraph',{
        sourceNode: source,
        targetNode: dest,
        k: 5,
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    UNWIND relationships(path) as r
    WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, r
    MATCH (n)-[f:FLEW_TO]->(m)
    WHERE n.code = startNode(r).code AND m.code = endNode(r).code
    RETURN
        index,
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost AS totalDistance,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS nodeNames,
        costs as distances,
        ROUND(SUM(DISTINCT(f.value)), 2) AS value,
        ROUND(SUM(DISTINCT(f.duration)), 3) AS duration
    ORDER BY index
''')

,index,sourceNodeName,targetNodeName,totalDistance,nodeNames,distances,value,duration
0,0,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1198.3,"[DFW, YYZ]","[0.0, 1198.3]",838.81,2.397
1,1,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.5,"[DFW, DTW, YYZ]","[0.0, 985.8, 1199.5]",839.65,2.399
2,2,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.7,"[DFW, IND, YYZ]","[0.0, 760.6, 1199.7]",839.79,2.399
3,3,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1200.0,"[DFW, FWA, DTW, YYZ]","[0.0, 858.5, 986.3, 1200.0]",840.00,2.400
4,4,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1203.2,"[DFW, STL, YYZ]","[0.0, 550.1, 1203.2]",842.24,2.406


#### Shortest Path with Duration

In [83]:
# Create projection to find shortest path with duration
# Used the aggregation of 'MIN' to get the paths into a single

gds.run_cypher('''
    CALL gds.graph.project(
        'shortestPathGraphDuration',
        'Airport',
        {
            FLEW_TO: {
                properties:{
                    duration: {
                        aggregation: 'MIN'
                    }
                }
            }
        }
    )
    YIELD
        graphName AS graph,
        relationshipProjection AS readProjection,
        nodeCount AS nodes,
        relationshipCount AS rels
''')

,graph,readProjection,nodes,rels
0,shortestPathGraphDuration,"{'FLEW_TO': {'orientation': 'NATURAL', 'indexInverse': False, 'aggregation': 'DEFAULT', 'type': 'FLEW_TO', 'properties': {'duration': {'defaultValue': None, 'property': 'duration', 'aggregation': 'MIN'}}}}",6072,37042


In [125]:
# Adding in value and duration as items in the results
gds.run_cypher('''
    MATCH (source:Airport {code: 'DFW'}), (dest:Airport {code: 'YYZ'})
    CALL gds.shortestPath.yens.stream('shortestPathGraph',{
        sourceNode: source,
        targetNode: dest,
        k: 5,
        relationshipWeightProperty: 'distance'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    UNWIND relationships(path) as r
    WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, r
    MATCH (n)-[f:FLEW_TO]->(m)
    WHERE n.code = startNode(r).code AND m.code = endNode(r).code
    RETURN
        index,
        gds.util.asNode(sourceNode).name AS sourceNodeName,
        gds.util.asNode(targetNode).name AS targetNodeName,
        totalCost AS totalDistance,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS nodeNames,
        costs as distances,
        ROUND(SUM(DISTINCT(f.value)), 2) AS value,
        ROUND(SUM(DISTINCT(f.duration)), 3) AS duration
    ORDER BY index
''')

,index,sourceNodeName,targetNodeName,totalDistance,nodeNames,distances,value,duration
0,0,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1198.3,"[DFW, YYZ]","[0.0, 1198.3]",838.81,2.397
1,1,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.5,"[DFW, DTW, YYZ]","[0.0, 985.8, 1199.5]",839.65,2.399
2,2,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1199.7,"[DFW, IND, YYZ]","[0.0, 760.6, 1199.7]",839.79,2.399
3,3,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1200.0,"[DFW, FWA, DTW, YYZ]","[0.0, 858.5, 986.3, 1200.0]",840.00,2.400
4,4,Dallas Fort Worth International Airport,Lester B. Pearson International Airport,1203.2,"[DFW, STL, YYZ]","[0.0, 550.1, 1203.2]",842.24,2.406


### Part 4: Adding the ability to add stops

In [16]:
# Get all possible paths with starting at DFW and with the input destinations

airports = ['IAH', 'YYZ', 'AMS']
permutations_list = list(permutations(airports))

orders = []

for order in permutations_list:
    orders.append(list(('DFW',) + order))

print(orders)


[['DFW', 'IAH', 'YYZ', 'AMS'], ['DFW', 'IAH', 'AMS', 'YYZ'], ['DFW', 'YYZ', 'IAH', 'AMS'], ['DFW', 'YYZ', 'AMS', 'IAH'], ['DFW', 'AMS', 'IAH', 'YYZ'], ['DFW', 'AMS', 'YYZ', 'IAH']]


In [17]:
# Get all the adjacent pairs - source and dest but in a single list

paths = []

for order in orders:
    path = []
    for i in range(len(order) - 1):
        pair = [order[i], order[i + 1]]
        path.append(pair)
    paths.append(path)

print(paths)


[[['DFW', 'IAH'], ['IAH', 'YYZ'], ['YYZ', 'AMS']], [['DFW', 'IAH'], ['IAH', 'AMS'], ['AMS', 'YYZ']], [['DFW', 'YYZ'], ['YYZ', 'IAH'], ['IAH', 'AMS']], [['DFW', 'YYZ'], ['YYZ', 'AMS'], ['AMS', 'IAH']], [['DFW', 'AMS'], ['AMS', 'IAH'], ['IAH', 'YYZ']], [['DFW', 'AMS'], ['AMS', 'YYZ'], ['YYZ', 'IAH']]]


In [18]:
# Loop through all possible routes and find shortest paths between them
dataframes = []

for j in range(len(paths)):
    for i in range(len(paths[0])):
        source = paths[j][i][0]
        dest = paths[j][i][1]
        result = gds.run_cypher('''
            MATCH (source:Airport {code: $source}), (dest:Airport {code: $dest})
            CALL gds.shortestPath.yens.stream('shortestPathGraph',{
                sourceNode: source,
                targetNode: dest,
                k: 2,
                relationshipWeightProperty: 'distance'
            })
            YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
            WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, path
            UNWIND relationships(path) as r
            WITH index, sourceNode, targetNode, totalCost, nodeIds, costs, path, r
            MATCH (n)-[f:FLEW_TO]->(m)
            WHERE n.code = startNode(r).code AND m.code = endNode(r).code
            RETURN
                totalCost AS totalDistance,
                [nodeId IN nodeIds | gds.util.asNode(nodeId).code] AS nodeNames,
                ROUND(SUM(DISTINCT(f.value)), 2) AS value,
                ROUND(SUM(DISTINCT(f.duration)), 3) AS duration
        ''', 
        params = {'source': source, 'dest': dest}
        )

        # Convert the result into a pandas dataframe
        df = pd.DataFrame(result, columns=['totalDistance', 'nodeNames', 'value', 'duration'])
        
        # Append the dataframe to the list
        dataframes.append(df)

len(dataframes)



18

In [19]:
# set up dataframes
# Create dataframes dynamically in a loop
for i in range(len(dataframes)):
    globals()[f"df{i+1}"] = dataframes[i]


# Generate all possible combinations of values - this is where it stops being automated - BEWARE
combinations1 = list(itertools.product(df1.to_dict('records'), df2.to_dict('records'), df3.to_dict('records')))
combinations2 = list(itertools.product(df4.to_dict('records'), df5.to_dict('records'), df6.to_dict('records')))
combinations3 = list(itertools.product(df7.to_dict('records'), df8.to_dict('records'), df9.to_dict('records')))
combinations4 = list(itertools.product(df10.to_dict('records'), df11.to_dict('records'), df12.to_dict('records')))
combinations5 = list(itertools.product(df13.to_dict('records'), df14.to_dict('records'), df15.to_dict('records')))
combinations6 = list(itertools.product(df16.to_dict('records'), df17.to_dict('records'), df18.to_dict('records')))


# Create a new dataframe with the combinations
combined_df1 = pd.DataFrame([{'totalDistance': d1['totalDistance'] + d2['totalDistance'] + d3['totalDistance'],
                            'totalValue': d1['value'] + d2['value'] + d3['value'],
                            'totalDuration': d1['duration'] + d2['duration'] + d3['duration'],
                            'nodeNames': [d1['nodeNames'], d2['nodeNames'], d3['nodeNames']]} for d1, d2, d3 in combinations1])

combined_df2 = pd.DataFrame([{'totalDistance': d4['totalDistance'] + d5['totalDistance'] + d6['totalDistance'],
                            'totalValue': d4['value'] + d5['value'] + d6['value'],
                            'totalDuration': d4['duration'] + d5['duration'] + d6['duration'],
                             'nodeNames': [d4['nodeNames'], d5['nodeNames'], d6['nodeNames']]} for d4, d5, d6 in combinations2])

combined_df3 = pd.DataFrame([{'totalDistance': d7['totalDistance'] + d8['totalDistance'] + d9['totalDistance'],
                            'totalValue': d7['value'] + d8['value'] + d9['value'],
                            'totalDuration': d7['duration'] + d8['duration'] + d9['duration'],
                             'nodeNames': [d7['nodeNames'], d8['nodeNames'], d9['nodeNames']]} for d7, d8, d9 in combinations3])

combined_df4 = pd.DataFrame([{'totalDistance': d10['totalDistance'] + d11['totalDistance'] + d12['totalDistance'],
                            'totalValue': d10['value'] + d11['value'] + d12['value'],
                            'totalDuration': d10['duration'] + d11['duration'] + d12['duration'],
                             'nodeNames': [d10['nodeNames'], d11['nodeNames'], d12['nodeNames']]} for d10, d11, d12 in combinations4])

combined_df5 = pd.DataFrame([{'totalDistance': d13['totalDistance'] + d14['totalDistance'] + d15['totalDistance'],
                            'totalValue': d13['value'] + d14['value'] + d15['value'],
                            'totalDuration': d13['duration'] + d14['duration'] + d15['duration'],
                             'nodeNames': [d13['nodeNames'], d14['nodeNames'], d15['nodeNames']]} for d13, d14, d15 in combinations5])

combined_df6 = pd.DataFrame([{'totalDistance': d16['totalDistance'] + d17['totalDistance'] + d18['totalDistance'],
                            'totalValue': d16['value'] + d17['value'] + d18['value'],
                            'totalDuration': d16['duration'] + d17['duration'] + d18['duration'],
                             'nodeNames': [d16['nodeNames'], d17['nodeNames'], d18['nodeNames']]} for d16, d17, d18 in combinations6])


# combine all dataframes into a single dataframe
all_dataframes = [combined_df1, combined_df2, combined_df3, combined_df4, combined_df5, combined_df6]

all_routes = pd.concat(all_dataframes, ignore_index = True)
all_routes.to_csv('routes/all_routes_results.csv')
all_routes




,totalDistance,totalValue,totalDuration,nodeNames
0,5226.6,3658.62,10.453,"[[DFW, IAH], [IAH, YYZ], [YYZ, AMS]]"
1,5231.8,3662.26,10.463,"[[DFW, IAH], [IAH, YYZ], [YYZ, MAN, AMS]]"
2,5230.4,3661.28,10.461,"[[DFW, IAH], [IAH, IND, YYZ], [YYZ, AMS]]"
3,5235.6,3664.92,10.471,"[[DFW, IAH], [IAH, IND, YYZ], [YYZ, MAN, AMS]]"
4,5240.1,3668.07,10.480,"[[DFW, CLL, IAH], [IAH, YYZ], [YYZ, AMS]]"
5,5245.3,3671.71,10.490,"[[DFW, CLL, IAH], [IAH, YYZ], [YYZ, MAN, AMS]]"
6,5243.9,3670.73,10.488,"[[DFW, CLL, IAH], [IAH, IND, YYZ], [YYZ, AMS]]"
7,5249.1,3674.37,10.498,"[[DFW, CLL, IAH], [IAH, IND, YYZ], [YYZ, MAN, AMS]]"
8,8948.1,6263.67,17.896,"[[DFW, IAH], [IAH, AMS], [AMS, YYZ]]"
9,8953.3,6267.31,17.906,"[[DFW, IAH], [IAH, AMS], [AMS, MAN, YYZ]]"


In [15]:
dataframes

[   totalDistance        nodeNames   value  duration
 0          224.7       [DFW, IAH]  157.29     0.449
 1          238.2  [DFW, CLL, IAH]  166.74     0.476,
    totalDistance        nodeNames   value  duration
 0         1280.1       [IAH, YYZ]  896.07     2.560
 1         1283.9  [IAH, IND, YYZ]  898.73     2.568,
    totalDistance        nodeNames    value  duration
 0         3721.8       [YYZ, AMS]  2605.26     7.444
 1         3727.0  [YYZ, MAN, AMS]  2608.90     7.454,
    totalDistance             nodeNames    value  duration
 0         5565.0            [AMS, LAX]  3895.50    11.130
 1         5612.9  [AMS, MAN, LAS, LAX]  3929.03    11.226,
    totalDistance        nodeNames   value  duration
 0          224.7       [DFW, IAH]  157.29     0.449
 1          238.2  [DFW, CLL, IAH]  166.74     0.476,
    totalDistance        nodeNames   value  duration
 0         1280.1       [IAH, YYZ]  896.07     2.560
 1         1283.9  [IAH, IND, YYZ]  898.73     2.568,
    totalDistance  